# Задание
Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
Cохранить результаты в pandas dataframe

Проверить состояние датафрейма и привести его в формат, позволяющий дальнейшее исследование данных: -- Привести колонки датафрейма к одному типу данных -- Проверить индекс, чтобы он соответствовал порядку строк -- Проверить наличие пропусков и в случае обнаружения, определить стратегию их обработки

Провести мини-исследование рынка строящейся недвижимости в одном или нескольких регионах с помощью pandas, matplotlib, seaborn, plotly и других инструментов.

-- Рассчитать средние цены за квадратный метр по регионам. Важно чтобы пропуски не учитывались при расчете средней цены. 

-- Рассчитать объемы вводимого жилья по регионам и по годам. 

-- Сравнить этажность домов, количество квартир, жилую площадь, количество парковочных мест и цену за квадратный метр в разных регионах 

-- Визуализировать разброс, связь и распределение этих показателей по регионам 

-- Сделать выводы по каждому из расчетов и графиков

In [8]:
#Импортирование библиотек
import requests
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
missing_values = ["nan",'N/A']
df = pd.read_csv('our_home_data.csv', na_values = missing_values)
pd.set_option('display.max_columns', None)

# Просмотр DataSet

In [ ]:
df.shape

In [ ]:
df.head(50)

In [ ]:
df.dtypes

In [ ]:
df.describe()

# Чистка

Удаление стобцов, где находится более 50% пустых ячеек

In [10]:
df = df.drop(columns = 'data.developer.bankruptStage.bankruptStageCd')
df = df.drop(columns = 'data.developer.bankruptStage.bankruptStageDesc')
df = df.drop(columns = 'data.developer.bankruptStage.bankruptLawUrl')
df = df.drop(columns = 'data.developer.orgBankruptMsgDttm')
df = df.drop(columns = 'data.metro.id')
df = df.drop(columns = 'data.metro.name')
df = df.drop(columns = 'data.metro.line')
df = df.drop(columns = 'data.metro.color')
df = df.drop(columns = 'data.metro.time')
df = df.drop(columns = 'data.metro.isWalk')
df = df.drop(columns = 'data.metro.colors')
df = df.drop(columns = 'data.transportDistIndexValue')
df = df.drop(columns = 'data.photoRenderDTO')
df = df.drop(columns = 'data.objectTransportInfo')

Очистка строк с NaN из-за невозможности восстановления данных для подзадачи 1

In [16]:
df_avg_P = df.dropna(subset=['data.objPriceAvg','data.developer.regRegionDesc'])

Также можно использовать очистку всех строк, где имеются NaN

In [ ]:
df.dropna(axis=0,inplace=True, how='all')

# Анализ Данных

### Рассчитать средние цены за квадратный метр по регионам. Важно чтобы пропуски не учитывались при расчете средней цены.

In [23]:
a = df.groupby('data.developer.regRegionDesc').mean()

In [26]:
a['data.objPriceAvg'].head(5)

data.developer.regRegionDesc
Адыгея Республика         59190.950000
Алтай Республика          58022.000000
Алтайский край            70012.280488
Амурская область         111318.333333
Архангельская область     86164.833333
Name: data.objPriceAvg, dtype: float64

### Рассчитать объемы вводимого жилья по регионам и по годам

# Возможно будет нужно

Регион - data.developer.regRegionDesc
Средняя цена по региону - data.objPriceAvg

Восстановление данных для подсчета цены за кв.м в разных регионах не является корректной.


In [6]:
df['data.objPriceAvg'].fillna(df['data.objPriceAvg'].median(), inplace=True)